- Compares fairness metrics methodologies

In [ ]:
import os

print(os.getcwd())

In [ ]:
# Pull in list of model performance metrics
import glob
import pandas as pd
file_list = glob.glob("*/*.csv")

file_list = [f for f in file_list if 'Fairness_metrics_comparing_subgroups' in f]
file_list

In [ ]:
file_dict = {}
for file_name in file_list:

    file = pd.read_csv(file_name, index_col = 0) 
    file_name = file_name.replace("Fairness_metrics_comparing_subgroups_", "")
    file_name = file_name.replace(".csv", "")
    LA, model_id = file_name.split('/')
    file['LA'] = LA
    file['model_id'] = model_id
    # Subgroup was index
    if (LA == 'LA2') or (LA == 'LA3'):
        file.reset_index(inplace = True, drop = False)
    file_dict[file_name] = file

print(file_dict.keys())

results = pd.concat(file_dict.values(), axis = 0, ignore_index = True)

In [ ]:
LA3 = pd.read_csv('Output/Fairness comparison LA3.csv', index_col = 0)
LA3.head()
LA3['LA'] = 'LA3'
LA3.drop(columns = ['value', 'Group'], inplace = True)

results = results.merge(LA3, how = 'outer', 
                            on = ['LA', 'model_id', 'Subgroup 1', 'Subgroup 2',
                            'Subgroup 1: Average precision score 95% CI (LL)',
                           'Subgroup 1: Average precision score 95% CI (UL)',
                           'Subgroup 2: Average precision score 95% CI (LL)',
                           'Subgroup 2: Average precision score 95% CI (UL)',
                           'Subgroups are significantly different according to a comparison of confidence intervals'])
print(results.shape)
print(LA3.shape)
print(results.shape)
# No duplicates by group
results.drop_duplicates(subset = ['LA', 'model_id', 'Subgroup 1', 'Subgroup 2']).shape ==results.drop_duplicates().shape


In [ ]:
# Check which are missing (see below)
pd.crosstab(results['LA'], results['model_id'])


In [ ]:
LA4 = pd.read_csv('Output/Fairness comparison LA4.csv', index_col = 0)
LA4.head()
LA4['LA'] = 'LA4'
LA4.drop(columns = ['value', 'Group'], inplace = True)

results = results.merge(LA4, how = 'outer', 
                            on = ['LA', 'model_id', 'Subgroup 1', 'Subgroup 2',
                            'Subgroup 1: Average precision score 95% CI (LL)',
                           'Subgroup 1: Average precision score 95% CI (UL)',
                           'Subgroup 2: Average precision score 95% CI (LL)',
                           'Subgroup 2: Average precision score 95% CI (UL)',
                           'Subgroups are significantly different according to a comparison of confidence intervals'])
print(results.shape)
print(LA4.shape)
print(results.shape)

In [ ]:
# Check which are missing (see below)
pd.crosstab(results['LA'], results['model_id'])

In [ ]:
# Differing numbers of observations comes from some models have missing disabilities and missing ages
pd.crosstab(results.loc[results['LA'] == 'LA3','Subgroup 1'], results.loc[results['LA'] == 'LA3','model_id'])

In [ ]:
pd.crosstab(results.loc[results['LA'] == 'LA3','Subgroup 2'], results.loc[results['LA'] == 'LA3','model_id'])

In [ ]:
import numpy as np
results['Subgroup 1'] = np.where((results['Subgroup 1']=='Ethnicity Not Known') |
                                    (results['Subgroup 1']=='Declined / Missing') ,
                                    'Ethnicity Not Known', results['Subgroup 1'])

results['Subgroup 1'] = np.where((results['Subgroup 1']=='Black / British Black') |
                                    (results['Subgroup 1']=='Black / African / Caribbean /Black British') |
                                    (results['Subgroup 1']=='Black/Black British'),
                                    'Black / African / Caribbean / Black British', results['Subgroup 1'])

results['Subgroup 1'] = np.where((results['Subgroup 1']=='Unknown, Unborn or Indeterminate') |
                                     (results['Subgroup 1']=='Unborn'),
                                    'Unknown, Unborn or Indeterminate', results['Subgroup 1'])

results['Subgroup 1'] = np.where((results['Subgroup 1']=='Asian British') |
                                     (results['Subgroup 1']=='Asian/Asian British'),
                                    'Asian / Asian British', results['Subgroup 1'])


In [ ]:
import numpy as np
results['Subgroup 2'] = np.where((results['Subgroup 2']=='Ethnicity Not Known') |
                                    (results['Subgroup 2']=='Declined / Missing') ,
                                    'Ethnicity Not Known', results['Subgroup 2'])

results['Subgroup 2'] = np.where((results['Subgroup 2']=='Black / British Black') |
                                    (results['Subgroup 2']=='Black / African / Caribbean /Black British') |
                                    (results['Subgroup 2']=='Black/Black British'),
                                    'Black / African / Caribbean / Black British', results['Subgroup 2'])

results['Subgroup 2'] = np.where((results['Subgroup 2']=='Unknown, Unborn or Indeterminate') |
                                     (results['Subgroup 2']=='Unborn'),
                                    'Unknown, Unborn or Indeterminate', results['Subgroup 2'])

results['Subgroup 2'] = np.where((results['Subgroup 2']=='Asian British') |
                                     (results['Subgroup 2']=='Asian/Asian British'),
                                    'Asian / Asian British', results['Subgroup 2'])


In [ ]:
# Some are NA because we merged in the LA3 and LA4 gender data which didn't have these values

results.loc[results["P-value lower than the significance threshold according to Hochberg's step up procedure"].notna(),
           "P-value lower than the significance threshold according to Hochberg's step up procedure"].value_counts(normalize=True)



In [ ]:
results['Subgroups are significantly different according to a comparison of confidence intervals'].value_counts(normalize=True, dropna = False)

In [ ]:
results['Subgroups are significantly different according to a comparison of confidence intervals'].value_counts()

In [ ]:
results['Subgroups are significantly different according to a comparison of confidence intervals'].replace({True: 1, False:0}, inplace = True)

In [ ]:
subgroup1 = results.groupby('Subgroup 1')['Subgroups are significantly different according to a comparison of confidence intervals'].mean()
subgroup2 = results.groupby('Subgroup 2')['Subgroups are significantly different according to a comparison of confidence intervals'].mean()
results_grouped_subgroup = pd.concat([subgroup1,subgroup2], axis = 1)
results_grouped_subgroup.columns = ['Sig and Subgroup 1', 'Sig and Subgroup 2']
results_grouped_subgroup.fillna(0, inplace = True)
results_grouped_subgroup['Proportion sig'] = round((results_grouped_subgroup['Sig and Subgroup 1'] +
                                              results_grouped_subgroup['Sig and Subgroup 2']) * 100, 0)
                                              

In [ ]:
results_grouped_subgroup['Characteristic'] = results_grouped_subgroup.index
results_grouped_subgroup['Characteristic'] = pd.Categorical(results_grouped_subgroup['Characteristic'],
                                                 categories = ['Under 1 Year', '1-4 Years', '5-9 Years', '10-15 Years',
                                   '16+ Years', 'Missing age', 'Female', 'Male',
                                   'Unknown, Unborn or Indeterminate', 'Disabled', 'Not Disabled',
                                   'Missing Disability', 'Asian / Asian British',
                                   'Black / African / Caribbean / Black British', 'Mixed Ethnicity',
                                   'Other Ethnicity', 'Ethnicity Not Known', 'White'], ordered = True)
results_grouped_subgroup.sort_values(by = 'Characteristic', inplace = True)

In [ ]:
results_grouped_subgroup = results_grouped_subgroup[['Characteristic', 'Proportion sig']]
results_grouped_subgroup.reset_index(drop = True, inplace = True)
results_grouped_subgroup.to_csv('Output/Subgroups which are significantly different by CI overlapping.csv')

## Data for visualisation

In [ ]:
results_for_model_summary = results[['LA', 'Subgroup 1', 'Subgroup 1: Average precision score 95% CI (UL)', 'Subgroup 2',
       'Subgroup 2: Average precision score 95% CI (LL)',
       'Subgroup 2: Average precision score 95% CI (UL)',
       'Subgroups are significantly different according to a comparison of confidence intervals',
       'model_id']]

In [ ]:
results_for_model_summary['Category'] = np.where(results_for_model_summary['Subgroup 1'].isin(['1-4 Years', '5-9 Years', '10-15 Years', '16+ Years', 'Missing age']),'Age',
                                                np.where(results_for_model_summary['Subgroup 1'].isin(['Male', 'Unknown, Unborn or Indeterminate']),'Gender',
                                                  np.where(results_for_model_summary['Subgroup 1'].isin(['Missing Disability','Not Disabled']), 'Disability',
                                                  np.where(results_for_model_summary['Subgroup 1'].isin(['Black / African / Caribbean / Black British','Ethnicity Not Known', 'Mixed Ethnicity', 'Other Ethnicity','White']), 'Ethnicity',np.nan))))
                                                

In [ ]:
pd.crosstab(results_for_model_summary['Category'], results_for_model_summary['Subgroup 1'])

In [ ]:
pd.crosstab(results_for_model_summary['Category'], results_for_model_summary['Subgroup 2'])

In [ ]:
results_for_model_summary_by_model_by_group = results_for_model_summary.groupby(['LA', 'model_id', 'Category'])['Subgroups are significantly different according to a comparison of confidence intervals'].max()

In [ ]:
results_for_model_summary_by_model_by_group = results_for_model_summary_by_model_by_group.reset_index()

In [ ]:
pd.crosstab(results_for_model_summary_by_model_by_group['LA'], results_for_model_summary_by_model_by_group['model_id'])

In [ ]:
results_for_model_summary_by_model_by_group.loc[(results_for_model_summary_by_model_by_group['LA'] == 'LA4') &
                                               (results_for_model_summary_by_model_by_group['model_id'] == 'rq2_ts_all'),]

In [ ]:
model_id = results_for_model_summary_by_model_by_group['model_id'].str.split('_', expand = True)
results_for_model_summary_by_model_by_group['rq'] = model_id[0]
results_for_model_summary_by_model_by_group['Validation'] = model_id[1]
results_for_model_summary_by_model_by_group['Data included'] = model_id[2]

In [ ]:
# Get name
question_names = pd.read_csv("Years and Sample Sizes.csv")
question_names['Prediction Number'] = np.where((question_names['Local authority'] == 'LA1')
                                               & (question_names['Research question'] == 'rq1'),1,
                                        np.where((question_names['Local authority'] == 'LA1')
                                               & (question_names['Research question'] == 'rq2'), 2,
                                        np.where((question_names['Local authority'] == 'LA2')
                                               & (question_names['Research question'] == 'rq1'),3,
                                        np.where((question_names['Local authority'] == 'LA2')
                                               & (question_names['Research question'] == 'rq2'), 4, 
                                        np.where((question_names['Local authority'] == 'LA3')
                                               & (question_names['Research question'] == 'rq1'),5,
                                        np.where((question_names['Local authority'] == 'LA3')
                                               & (question_names['Research question'] == 'rq2'), 6,
                                         np.where((question_names['Local authority'] == 'LA4')
                                               & (question_names['Research question'] == 'rq1'),7, 8)))))))  
final_fairness = results_for_model_summary_by_model_by_group.merge(question_names[['Local authority', 'Research question', 'Shortened outcome', 'Prediction Number']], 
                                                  how = 'left', left_on = ['LA', 'rq'], right_on = ['Local authority', 'Research question'])

In [ ]:
final_fairness = final_fairness[['Local authority', 'Prediction Number', 'Shortened outcome', 'Validation', 'Data included', 'Category',                                  
                                 'Subgroups are significantly different according to a comparison of confidence intervals']]


In [ ]:
final_fairness['Validation'] = final_fairness['Validation'].replace({'ss': 'Learned from all cases', 'ts': 'Learned only from earlier cases'})
final_fairness['Data included'] = final_fairness['Data included'].replace({'str': 'Structured data only', 'all': 'Includes text data'})

In [ ]:
final_fairness.groupby(['Shortened outcome', 'Validation', 'Data included', 'Category'])['Subgroups are significantly different according to a comparison of confidence intervals'].max()

In [ ]:
print(final_fairness.shape)
final_fairness.drop_duplicates(inplace = True)
final_fairness.dropna(axis = 0, subset = ['Category'], inplace = True)
print(final_fairness.shape)

In [ ]:
final_fairness.to_csv('Bias metrics.csv', index = False)